# Lab: Fisher Information Matrix and Profile Likehihood
### Jessica Conrad, MSPH
### MSRI Summer School on Algebraic Geometry July 2022 (Part 2)

#### Based off of the Parameter Estimation Lab by Dr. Marisa Eisenberg found [here](https://epimath.org/epid-814-materials/Labs/EstimationLab/) and [here](https://epimath.org/epid-814-materials/Labs/IdentifiabilityUncertainty/IdentifiablilityUncertaintyLab.html)

# Part 1: Recall our model

Here we resume our exploration of the following model:
$$
\begin{align}
\frac{dS}{dt} & = \mu -\beta SI - \mu S\\     
\frac{dI}{dt} & = \beta SI - \gamma I -\mu I \\ 
\frac{dR}{dt} & = \gamma I - \mu R,             
\end{align}
$$
with measurement equation $y = k I$. In this case we are using the reparameterized model where we use the following definition:
* **S(t)** : susceptible fraction of the population at time *t*
* **I(t)** : infected fraction of the population at time *t*
* **R(t)** : removed fraction of the population at time *t*
* __$\beta$__ : infection rate; expected number of secondary infections per time *t*
* __$\gamma$__ : recovery rate; $\frac{1}{\gamma}$ is average time a person is infectious/infected
* __$\mu$__ : death rate

We assumed the birth and death rates are slow enough to assume $\mu = 0$. This assumption is only valid for fast acting diseases.
As such, let's define our "true" parameter set such that $\beta = 0.4$, $\gamma = 0.25$, and $\kappa = 80000.$

Ok then we will provide for you the code we generated last session:

In [15]:
#### Import the relevant libraries ####
from math import *                         # useful math functions
import numpy as np                         # useful array objects 
                                           # (also a core scientific computing library)

from scipy.integrate import odeint as ode  # ode solver
import matplotlib.pyplot as plt            # nice plotting commands, very similar to Matlab commands
from scipy.stats import poisson
from scipy.stats import norm
import scipy.optimize as optimize          #optimizer function package

#### Redefine the functions we wrote ####
def model(ini, time_step, params):
    #Define our ODE model
    
    ###########################
    #Input:
    #   ini           initial inputs for the state variables S, I, R
    #   time_step     time step definition; used by the ODE wrapper
    #   params        parameters for this model, in this case beta and gamma
    #
    #Output:
    #   Y             vector of state variable equations for S, I, and R
    ###########################
	Y = np.zeros(3) #column vector for the state variables
	X = ini
	mu = 0
	beta = params[0]
	gamma = params[1]

	Y[0] = mu - beta*X[0]*X[1] - mu*X[0]         
	Y[1] = beta*X[0]*X[1] - gamma*X[1] - mu*X[1] 
	Y[2] = gamma*X[1] - mu*X[2]                  

	return Y


def x0fcn(params, data):
    #Set initial conditions
    
    ###########################
    #Input:
    #   data          true data to be fit
    #   params        parameters for this mode, in this case beta, gamma, and kappainv
    #
    #Output:
    #   X0            initial conditions for the SIR ODE          
    ###########################
	S0 = 1.0 - (data[0]/params[2]) 
	I0 = data[0]/params[2]         
	R0 = 0.0                       
	X0 = [S0, I0, R0]

	return X0


def yfcn(res, params):
    #Define measurement equation
    
    ###########################
    #Input:
    #   res           simulated data results
    #   params        parameters for this mode, in this case beta, gamma, and kappainv
    #
    #Output:
    #   simulated reported data 
    ###########################
	return res[:,1]*params[2]


def NLL(params, data, times): 
    #Define the negative log likelihood
    
    ###########################
    #Input:
    #   params        parameters for this mode, in this case beta, gamma, and kappainv
    #   data          true data to be fit
    #   times         time points when the true data is recorded
    #
    #Output:
    #   nll           negative log likelihood estimate    
    ###########################
	params = np.abs(params)
	data = np.array(data)
    
    #Simulate the model with current parameters
	res = ode(model, x0fcn(params,data), times, args=(params,))
    
    #Apply the measurement equation
	y = yfcn(res, params)
    
    #Calculate the NLL for Poisson distribution
	nll = sum(y) - sum(data*np.log(y))   #(****remove for sandbox version of code****)
    
	# note this is a slightly shortened version--there's an additive constant term missing but it 
	# makes calculation faster and won't alter the threshold. Alternatively, can do:
	# nll = -sum(np.log(poisson.pmf(np.round(data),np.round(y)))) # the round is b/c Poisson is for (integer) count 
	# data this can also barf if data and y are too far apart because the dpois will be ~0, which makes the log 
    # angry
	
	# ML using normally distributed measurement error (least squares)
	# nll = -sum(np.log(norm.pdf(data,y,0.1*np.mean(data)))) # example WLS assuming sigma = 0.1*mean(data)
	# nll = sum((y - data)**2)  # alternatively can do OLS but note this will mess with the thresholds 
	#                             for the profile! This version of OLS is off by a scaling factor from
	#                             actual LL units.
	return nll

In [ ]:
#### Load Data ####
times = [0, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98]
data = [97, 271, 860, 1995, 4419, 6549, 6321, 4763, 2571, 1385, 615, 302, 159, 72, 34]

#### Define our parameter set ####
params = [0.4, 0.25, 80000.0]        #make sure all the params and inition states are float
paramnames = ['beta', 'gamma', 'k']

#### Simulate the model ####
ini = x0fcn(params,data)
res = ode(model, ini, times, args=(params,))
sim_measure = yfcn(res, params)

#### Plot the data and simulation ####
plt.plot(times, sim_measure, 'b-', linewidth=3, label='Model simulation')
plt.plot(times, data, 'ko', linewidth=2, label='Data')
plt.xlabel('Time')
plt.ylabel('Individuals')
plt.legend()
plt.show()

<div class="alert alert-block alert-warning">
    <b>Python Hint:</b> Why did I separate the coding blocks that contain <i>libraries and function definitions</i> versus <i> parameter settings and function calls</i>? Generally it is good coding practice to write your functions in a file separate from variables and parameters that you the user will change. This will help prevent user errors. We first call our libraries and functions, and only then do we run our code experiments. 
</div>

Let's also take a moment to remember how the likelihood function `NLL` could be used to estimate our parameters by wrapping it in an optimizer!

In [ ]:
#### Choose an optimizer and estimate parameter values ####
optimizer = optimize.minimize(NLL, params, args=(data, times), method='Nelder-Mead')
paramests = np.abs(optimizer.x)

#### Re-generate initial case data based on new parameter estimates
iniests = x0fcn(paramests, data)      

#### Re-simulate and plot the model with the final parameter estimates ####
#Simulate data
xest = ode(model, iniests, times, args=(paramests,))    
#Apply the measurement equation
est_measure = yfcn(xest, paramests)                     

#Plot
plt.plot(times, est_measure, 'b-', linewidth=3, label='Model simulation')
plt.plot(times, data, 'ko', linewidth=2, label='Data')
plt.xlabel('Time')
plt.ylabel('Individuals')
plt.legend()
plt.show()

print('The parameter estimates are beta = {params[0]:.4}, gamma = {params[1]:.4}, and kappa = {params[2]}'.format(params=paramests))

# Part 2: Fisher Information Matrix

Ok we are all caught up on where we left off. Let's set up a new function to calculate the Fisher Information Matrix. Recall from the lecture, the exact definition of the Fisher Information Matrix is:
$$
I(p)_{ij}= \mathop{\mathbb{E}}[(\frac{\partial}{\partial p_i} log⁡ \mathcal{L}(z,p))(\frac{\partial}{\partial p_j}   log⁡ \mathcal{L}(z,p))| p]
$$

How do we translate this into code? The expectation function is going to be dependent on your model and likelihood function $\mathcal{L}(z,p)$, so we need to write code specific to the SIR model. We are not going to walk through this particular derivation and instead we will spend time learning how to make sense of the results.


<div class="alert alert-block alert-info">
<b>Note:</b> The FIM is a statistical method dependent on the quality of your data $z$, where z is defined as $z_i = y_i + e_i$, where $y$ is the measurement equation and $e$ is the error in each measurement of that output. If you are unable to characterize or limit the noise in your ouput data $z$, then all you will get is nonsense. As the age old saying goes: Garbage in, garbage out.
</div>

### Here is the FIM function for the SIR model:

In [9]:
# Simplified FIM (Fisher information matirx) function for the SIR model
# Marisa Eisenberg (marisae@umich.edu)
# Yu-Han Kao (kaoyh@umich.edu) -7-9-17

def minifisher (times, params, data, delta = 0.001):
    #Calculate the FIM for the SIR model.
    
    ###########################
    #Input:
    #   times         time points when the true data is collected
    #   params        parameters for this mode, in this case beta, gamma, and kappainv
    #   data          true data to be fit
    #   delta         fit parameter for FIM; preset to 0.001, but can be set by user
    #
    #Output:
    #   simulated reported data 
    ###########################
	#params = np.array(params)
	listX = []
	params_1 = np.array (params)
	params_2 = np.array (params)
	for i in range(len(params)):
		params_1[i] = params[i] * (1+delta)
		params_2[i]= params[i] * (1-delta)

		res_1 = ode(model, x0fcn(params_1,data), times, args=(params_1,))
		res_2 = ode(model, x0fcn(params_2,data), times, args=(params_2,))
		subX = (yfcn(res_1, params_1) - yfcn(res_2, params_2)) / (2 * delta * params[i])
		listX.append(subX.tolist())
	X = np.matrix(listX)
	FIM = np.dot(X, X.transpose())
	return FIM

<div class="alert alert-block alert-info">
<b>Note:</b> This FIM code is a numerical approximation method. Specifically, `MiniFisher` is using a numerical approximation for the derivative. Ariel Citr&oacute;n Arias's slides found <a href="http://www.nimbios.org/wordpress-training/parameter/wp-content/uploads/sites/14/2014/03/ols_sir_lecture.pdf">here</a> give a nice introduction to estimation and sensitivity equations using the forward sensitivity equations instead. 
</div>

Ok so now that we have our function, model, and data, let's try running this example and see what the FIM looks like and explore some of its features!

### Write a line of code to generate the FIM for our model by looking at how the function call was defined above. 

In [2]:
#### Calculate the simplified Fisher Information Matrix (FIM) ####


<div class="alert alert-block alert-warning">
    <b>Python Hint:</b> Are you wondering how to see what the objects you created actually look like? Try the `print` function. For example, after you generate the object called "FIM", in the next line write: `print(FIM)`. Click "Run" in the upper left corner and see what happens! 
</div>

### Using `np.linalg.matrix_rank()`, calculate the rank of your new matrix.

In [1]:
#### Calculate rank of FIM ####


<div class="alert alert-block alert-success">
<b>Let's Ponder:</b> What do we expect the structure of the FIM to look like? What does this tell us about the identifiability of the model? Try it out with the other un-scaled SIR model given in Lab (1) (or you can use the un-scaled SIR from the parameter estimation lab) -- how does that change things?
</div>

As a note to yourself, write down some of your findings and thoughts in the Markdown box below OR add new coding boxes to explore the suggested problems!

# Part 3: Generating Profile Likelihoods

The recovery rate $\gamma$ is often approximately known, so let’s fix the value of $\gamma=0.25$.
Now we have only two unknown parameters, $\beta$ and $\kappa$.
We want to plot the likelihood as a surface or heat map as a function of $\beta$ and $\kappa$ (i.e. so that color is the likelihood value, and your x and y axes are the $\beta$ and $\kappa$ values respectively. 

<div class="alert alert-block alert-info">
<b>Note:</b> Recall that $\gamma$ is the recovery rate, and $\frac{1}{\gamma}$ is the time spent in state $I$, or "time spent infected/infectious". Generally, a person is considered "infected" in our model when they present symptoms of disease. Symptoms of disease are noticeable, so we can often accurately approximate the "time spent infected/infectious" -- the inverse value of $\gamma$! Therefore, $\gamma$ is often known within some bound of uncertainty.
</div>

As an example, here’s some code to plot the likelihood for the Poisson case we used earlier. First, choose your own $\beta$ and $\gamma$ range of value to explore and their interval.

### Use `np.arange` to  define a sequence of values for $\beta$ and $\gamma$ that you want to itterate through. Use `np.zeros` to generate a blank matrix called "likevals" to store the likelihood values in.

In [ ]:
# Define the ranges for each parameter, and make an empty matrix for the likelihood values
...
...
...

print(betarange)
print(kapparange)

I have sketched out some "for" loops for you. 
### How do we populate each value of the "likevals" matrix using the `NLL` functions?

In [ ]:
# Go through each point on the contour plot and calculate the likelihood value at those coordinates
for i in range(len(betarange)):
    for j in range(len(kapparange)):
        likevals[i,j] = ... #Recal NLL(params, data, times)
        
print(np.min(likevals))
print(np.max(likevals))

Ok now that we have the Liklihood values for different $\beta$ and $\gamma$ combinations, let's plot this matrix as a contour to visually assess what our results are. You can try different ranges for beta and kappa depending on how far out you want to look at the plot!

### How does the shape of the likelihood change as you switch likelihood functions? 
It may not change much, but you can often notice small differences between likelihood choices. What does the likelihood landscape tell us about the parameter identifiability of this model, assuming $\gamma$ is known?

In [ ]:
#### Make a contour plot! #####
plt.contourf(betarange, kapparange, likevals)
plt.xlabel('Beta Range')
plt.ylabel('Kappa Range')
plt.colorbar()
plt.show()

### Please write a discussion of your findings in the Markdown box that follows.

# Part 4: Profile Likelihood to Confidence Bounds on Parameters

Now we have started to learn how a liklihood space might look if we fix one of our parameters and explore the combinations of other parameters. 
But in the lecture, we discussed looking at the likelihood plots of singular parameters when letting *all* parameters vary in value.

In this section, we will explore <b>Profile Likelihood Plots</b> and <b>Likelihood-based Confidence Intervals</b>.

To make our lives a little easier, I am not going to make you write the profile likelihood generator code. Instead, let's walk through the profile likelihood generator code and make sure that we have an understanding of what it does, and more importantly does what it claims to do! 

In [147]:
# Profile Likelihood Generator
# Marisa Eisenberg (marisae@umich.edu)
# Yu-Han Kao (kaoyh@umich.edu) -7-9-17

def profcost (fit_params, profparam, profindex, data, times, cost_func):
	paramstest = fit_params.tolist()
	paramstest.insert(profindex, profparam)
	return cost_func (paramstest, data, times)


# Input definitions
# params = starting parameters (all, including the one to be profiled)
# profparam = index within params for the parameter to be profiled
#   ---reminder to make this allow you to pass the name instead later on
# costfun = cost function for the model - should include params, times, and data as arguments.
#   Note costfun doesn't need to be specially set up for fixing the profiled parameter, 
#   it's just the regular function you would use to estimate all the parameters
#   (it will get reworked to fix one of them inside ProfLike)
# times, data = data set (times & values, or whatever makes sense)
#   ---possibly change this so it's included in costfun and not a separate set of inputs? Hmm.
# perrange = the percent/fraction range to profile the parameter over (default is 0.5)
# numpoints = number of points to profile at in each direction (default is 10)

# Output
# A list with:
#   - profparvals: the values of the profiled parameter that were used
#   - fnvals: the cost function value at each profiled parameter value
#   - convergence: the convergence value at each profiled parameter value
#   - paramestvals: the estimates of the other parameters at each profiled parameter value

def proflike (params, profindex, cost_func, times, data, perrange = 0.5, numpoints = 10):
	profrangedown = np.linspace(params[profindex], params[profindex] * (1 - perrange), numpoints).tolist()
	profrangeup = np.linspace(params[profindex], params[profindex] * (1 + perrange), numpoints).tolist()[1:] #skip the duplicated values
	profrange = [profrangedown, profrangeup]
	currvals = []
	currparams = []
	currflags = []

	fit_params = params.tolist() #make a copy of params so we won't change the origianl list
	fit_params.pop(profindex)
	print('Starting profile...')
	for i in range(len(profrange)):
		for j in profrange[i]:
			#print(i, j)
			optimizer = optimize.minimize(profcost, fit_params, args=(j, profindex, data, times, cost_func), method='Nelder-Mead')
			fit_params = np.abs(optimizer.x).tolist() #save current fitted params as starting values for next round
			#print(optimizer.fun)
			currvals.append(optimizer.fun)
			currflags.append(optimizer.success)
			currparams.append(np.abs(optimizer.x).tolist())

	#structure the return output
	profrangedown.reverse()
	out_profparam = profrangedown+profrangeup
	temp_ind = range(len(profrangedown))
	out_params = [currparams[i] for i in reversed(temp_ind)]+currparams[len(profrangedown):]
	out_fvals = [currvals[i] for i in reversed(temp_ind)]+currvals[len(profrangedown):]
	out_flags = [currflags[i] for i in reversed(temp_ind)]+currflags[len(profrangedown):]
	output = {'profparam': out_profparam, 'fitparam': np.array(out_params), 'fcnvals': out_fvals, 'convergence': out_flags}
	return output


Hopefully you wrote yourself notes in the code above. Now let's start by defining our confidence interval.

For the threshold to use in determining your confidence intervals, we note that $2(NLL(p)−NLL(\hat{p}))$ (where NLL is the negative log likelihood) is approximately $\chi^2$-distributed with degrees of freedom equal to the number of parameters fitted (including the profiled parameter). 
Then an approximate 95% (for example) confidence interval for $p$ can be made by taking all values of $p$ that lie within the 95$^{th}$ percentile range of the $\chi^2$-distribution for the given degrees of freedom.

In this case, for a 95% confidence interval, we have three total parameters we are estimating ($\beta, \gamma$, and $\kappa$), so the $\chi^2$ value for the 95$^{th}$ percentile is 7.8147. Then the confidence interval is any $p$ such that:
$$
NLL(p)\leq NLL(\hat{p})+7.8147/2
$$

In other words, our threshold is $NLL(\hat{p})+7.8147/2=NLL(\hat{p})+3.9074$, where $NLL(\hat{p})$ is the cost function (aka likelihood function) value at our parameter estimates from Part (1).

### Below we have provided code that automatically generates an appropriate confidence threshold.

In [148]:
#### Generate profile likelihoods and confidence bounds ####
import scipy.stats as stats

threshold = stats.chi2.ppf(0.95,len(paramests))/2.0 + optimizer.fun
perrange = 0.25 #percent range for profile to run across

Now that we can determine if our parameter estimate is reasonable using our threshold, let's try profiling the parameters with a profile likelihood plot, as seen in our earlier lectures. 

### Are the parameters practically idenitifiable? Why or why not?

In [ ]:
#### Plot the individual parameter profiles ####

profiles={}
for i in range(len(paramests)):
	profiles[paramnames[i]] = proflike(paramests, i, NLL, times, data, perrange=perrange)
	print('Profiles have been generated.')
	plt.figure()
	plt.scatter(paramests[i], optimizer.fun, marker='*',label='True value', color='k',s=150, facecolors='w', edgecolors='k')
	plt.plot(profiles[paramnames[i]]['profparam'], profiles[paramnames[i]]['fcnvals'], 'k-', linewidth=2, label='Profile likelihood')
	plt.axhline(y=threshold, ls='--',linewidth=1.0, label='Threshold', color='k')
	plt.xlabel(paramnames[i])
	plt.ylabel('Negative log likelihood')
	plt.legend(scatterpoints = 1)
	paramnames_fit = [ n for n in paramnames if n not in [paramnames[i]]]
	paramests_fit = [v for v in paramests if v not in [paramests[i]]]
	print(paramnames_fit)
	print(paramests_fit)
    
#print(profiles)
plt.show()

[Double click here. Fill in this Markdown box with your answers]

In [ ]:
#### Plot the parameter relationships ####

profiles={}
for i in range(len(paramests)):
	profiles[paramnames[i]] = proflike(paramests, i, NLL, times, data, perrange=perrange)
	print('Profiles have been generated.')
	paramnames_fit = [ n for n in paramnames if n not in [paramnames[i]]]
	paramests_fit = [v for v in paramests if v not in [paramests[i]]]
	for j in range(profiles[paramnames[i]]['fitparam'].shape[1]):
		plt.figure()
		plt.plot(profiles[paramnames[i]]['profparam'],profiles[paramnames[i]]['fitparam'][:,j],'k-', linewidth=2, label=paramnames_fit[j])
		plt.scatter(paramests[i], paramests_fit[j], marker='*',label='True value', color='k',s=150, facecolors='w', edgecolors='k')
		plt.xlabel(paramnames[i])
		plt.ylabel(paramnames_fit[j])
		plt.legend(scatterpoints = 1)
#print(profiles)
plt.show()

What do we learn from the 2 parameter relationship analysis?

# Part 5: Bonus! Back to that real life drama

Lastly, let us consider the case where you are attempting to fit and forecast an ongoing epidemic (i.e. with incomplete data). Truncate your data to only include the first seven data points (i.e. just past the peak), then re-fit the model parameters and generate the profile likelihoods with the truncated data (you can also see if truncating the data affects the FIM rank!).

* How do your parameter estimates change?

* Does the practical identifiability of the parameters change? How so?

* If any of the parameters were unidentifiable, examine the relationships between parameters that are generated in the profile likelihoods. Can you see any interesting relationships between parameters? What do you think might be going on—why has the identifiability changed?

Please add coding and markdown blocks below this section as you explore and 